<a href="https://colab.research.google.com/github/FilipeSquire/Integrations/blob/main/FB_API_Connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json 
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

# Etapa 1 - Dados Gerais

# Treatment

In [ ]:
#Vamos dar uma olhada em todas as contas que hoje estão ativas
events = requests.get("INSERT YOUR REQUEST HERE")
trans = json.loads(events.content)
contas = pd.json_normalize(trans['adaccounts']['data'])

In [ ]:
#Fazendo requisicao para graphi face api para listar todas as campanhas existentes

events = requests.get("INSERT YOUR URL HERE")
trans = json.loads(events.content)
redes = pd.json_normalize(trans['data'])


Abaixo vamos fazer o tratamento dos dados das campanhas

In [ ]:
#Requisitando as informações via url retirado do graph facebook api
events = requests.get("INSER YOUR URL HERE")
#A base vai vir com campaigns_id que não possui dados internos. Tratamento dos dados abaixo
trans = json.loads(events.content)
trans2 = pd.json_normalize(trans['data']) #os dados que queremos é insights.data
trans2 = trans2.dropna(subset=['insights.data'],inplace=False).reset_index() #removendo as linhas nulas
del trans2['index'] #tratando index

#Agora vamos criar um loop para preencher uma tabela com os dados que queremos
size = trans2['id'].count()

dataset2 = pd.DataFrame(columns=['account_id','campaign_id','date_start','date_stop','impressions','spend'])

for x in range(0,size): #o loop vai olhar dentro de cada campanha e armazenar os dados q queremos em uma tabela nova
  filtro = pd.json_normalize(trans2['insights.data'][x])
  dataset2 = pd.merge(dataset2, filtro, how='outer')


In [ ]:
#Agora vamos preparar o merge para sabermos os nomes das campanhas que estamos lidando

redes = redes.rename(columns={'id':'campaign_id'})
#Agora temos um dataset oficial e pronto pra ir pra o excel!
dts = pd.merge(dataset2,redes,on='campaign_id')

# **DEPLOY**

In [ ]:
#O nosso dataset é dts
!pip install --upgrade gspread
!pip install --upgrade gspread-pandas


In [ ]:
#Estabelecendo conexao com gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
#sh = gc.create('INSERT NEW SHEETS NAME')

#Openining the new sheet
worksheet = gc.open('INSERT SHEETS NAME')

plan1 = worksheet.worksheet('Redes Campaigns')
plan1.update([dts.columns.values.tolist()] + dts.values.tolist())

{'spreadsheetId': '15e96GL-KH-p27H7JjW3cPSPdvMY-qa_OfM9RJZCYPXg',
 'updatedCells': 27,
 'updatedColumns': 9,
 'updatedRange': "'Branding Campaigns'!A1:I3",
 'updatedRows': 3}

# Etapa 2 - Visão Diaria da Conta

In [ ]:
#Fazendo requisicao para graphi face api para listar todas as campanhas existentes
#Ao contrario das extrações anteriores, agora teremos que detalhar o quanto foi gasto diariamente por conta e por campanha

#Custos diarios de uma Conta
events = requests.get("INSERT CODE HERE")
trans = json.loads(events.content)
rd_daily = pd.json_normalize(trans['data'])



Agora vamos jogar essas informações na nossa planilha

In [ ]:
#Estabelecendo conexao com gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
#sh = gc.create('INSERT YOUR SHEETS HERE')

#Openining the new sheet
worksheet = gc.open('INSERT YOUR SHEETS HERE')

plan1 = worksheet.worksheet('Daily Account Costs')
plan1.update([rd_daily.columns.values.tolist()] + rd_daily.values.tolist())

{'spreadsheetId': '15e96GL-KH-p27H7JjW3cPSPdvMY-qa_OfM9RJZCYPXg',
 'updatedCells': 170,
 'updatedColumns': 5,
 'updatedRange': "'Branding Costs'!A1:E34",
 'updatedRows': 34}

# Etapa 3 - Visao Diaria de Campanha

In [ ]:
#Custos diarios das campanhas 

rd_daily = pd.DataFrame(columns=['spend','impressions','cpm','cpc','date_start','date_stop','campaign_id'])

for x in range(0,dts['campaign_id'].count()):
  campanha = dts['campaign_id'][x]
  events = requests.get("https://graph.facebook.com/v10.0/"+campanha+"REST OF THE URL HERE")
  trans3 = json.loads(events.content)
  filtro = pd.json_normalize(trans3['data'])
  rd_daily = pd.merge(rd_daily, filtro, how='outer')
  rd_daily['campaign_id'].fillna(dts['campaign_id'][x],inplace=True)

rd_daily = pd.merge(rd_daily,redes,on='campaign_id')

rd_daily


Pronto! Agora acabamos de listar todas as contas, os custos diarios das contas e custos diarios das campanhas e armazenamos tudo em um Sheets que criamos! É possivel fazer mais requisições, basta consultar o material do FB Graph API